## インストール


In [1]:
!pip install -q --no-warn-conflicts git+https://github.com/drillan/colab-pydantic-ai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.4/357.4 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 23.4 MB/s eta

## 使い方

### 基本的な使い方


In [1]:
from colab_pydantic_ai import ColabGeminiModel
from pydantic_ai import Agent

# モデルの作成（セットアップは自動で完了します）
model = ColabGeminiModel()
agent = Agent(model)

# Agentの実行
result = agent.run_sync("フランスの首都は？")
print(result.output)

フランスの首都は**パリ**です。


### モデルの選択

利用可能なモデルを確認：


In [2]:
from google.colab import ai

ai.list_models()

['google/gemini-2.5-flash', 'google/gemini-2.5-flash-lite']

特定のモデルを使用：


In [ ]:
from colab_pydantic_ai import ColabGeminiModel

# Gemini 2.5 Flash Liteを使用
model = ColabGeminiModel("google/gemini-2.5-flash-lite")

### 構造化出力

Pydantic AI の型安全な機能を活用できます：


In [ ]:
from colab_pydantic_ai import ColabGeminiModel
from pydantic_ai import Agent
from pydantic import BaseModel


class City(BaseModel):
    name: str
    country: str
    population: int


model = ColabGeminiModel()
agent = Agent(model, output_type=City)

result = agent.run_sync("東京について教えて")
city = result.output
print(f"{city.name}, {city.country}, 人口: {city.population:,}")

Tokyo, Japan, 人口: 14,000,000


## サンプル

### サンプル 1: リスト型フィールドを持つ構造化出力

リスト型フィールドを含むデータモデル


In [ ]:
from colab_pydantic_ai import ColabGeminiModel
from pydantic_ai import Agent
from pydantic import BaseModel, Field


class EmailAddress(BaseModel):
    email: str = Field(pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")
    domain: str


model = ColabGeminiModel()
agent = Agent(model, output_type=EmailAddress, retries=3)

# バリデーションエラーが発生した場合、自動的にリトライされる
result = agent.run_sync("example@gmail.comのメールアドレス情報を抽出してください")
email = result.output
print(f"メール: {email.email}")
print(f"ドメイン: {email.domain}")

### サンプル 2: システムプロンプト付き Agent

役割を持たせた Agent で専門的な回答を得る


In [ ]:
from colab_pydantic_ai import ColabGeminiModel
from pydantic_ai import Agent
from pydantic import BaseModel


class Book(BaseModel):
    title: str
    author_name: str
    year: int
    genres: list[str]


model = ColabGeminiModel()
agent = Agent(model, output_type=Book)

result = agent.run_sync("村上春樹の『ノルウェイの森』について教えて")
book = result.output
print(f"タイトル: {book.title}")
print(f"著者: {book.author_name}")
print(f"出版年: {book.year}")
print(f"ジャンル: {', '.join(book.genres)}")

### サンプル 3: 依存関係注入（Dependency Injection）

RunContext を使って動的な値を Agent に渡す


In [ ]:
from colab_pydantic_ai import ColabGeminiModel
from pydantic_ai import Agent

model = ColabGeminiModel()
agent = Agent(
    model,
    system_prompt="あなたは日本史の専門家です。歴史的な事実を正確に、わかりやすく説明してください。",
)

result = agent.run_sync("関ヶ原の戦いについて簡潔に説明してください")
print(result.output)

### サンプル 4: 温度パラメータの調整

temperature を変更して創造的/正確な出力を制御


In [ ]:
from colab_pydantic_ai import ColabGeminiModel
from pydantic_ai import Agent, RunContext
from dataclasses import dataclass


@dataclass
class UserProfile:
    name: str
    age: int
    interests: list[str]


model = ColabGeminiModel()
agent = Agent(
    model,
    deps_type=UserProfile,
    system_prompt="ユーザーのプロフィールに基づいて、パーソナライズされた提案をしてください。",
)


@agent.system_prompt
def add_user_context(ctx: RunContext[UserProfile]) -> str:
    return f"""
    ユーザー情報:
    - 名前: {ctx.deps.name}
    - 年齢: {ctx.deps.age}
    - 興味: {", ".join(ctx.deps.interests)}
    """


user = UserProfile(
    name="太郎", age=25, interests=["プログラミング", "機械学習", "読書"]
)
result = agent.run_sync("おすすめの学習リソースを3つ教えてください", deps=user)
print(result.output)

### サンプル 5: エラーハンドリングとリトライ

バリデーションエラー時の自動リトライ機能


In [ ]:
from colab_pydantic_ai import ColabGeminiModel
from pydantic_ai import Agent
from pydantic_ai.settings import ModelSettings

# 低温度（正確な出力）
model_precise = ColabGeminiModel(settings=ModelSettings(temperature=0.3))
agent_precise = Agent(model_precise)

# 高温度（創造的な出力）
model_creative = ColabGeminiModel(settings=ModelSettings(temperature=1.0))
agent_creative = Agent(model_creative)

prompt = "AIについて一文で説明してください"

print("【正確な出力 (temperature=0.3)】")
result1 = agent_precise.run_sync(prompt)
print(result1.output)
print()

print("【創造的な出力 (temperature=1.0)】")
result2 = agent_creative.run_sync(prompt)
print(result2.output)